In [1]:
from openai import OpenAI, AsyncOpenAI
import asyncio
import os

In [2]:
from pydantic_classes import FinalAnswer
from prompts import prompts



In [3]:
print(prompts["emotional_qual"])


IMPORTANT: This is not just another MCQ - a real human life depends on your answer! 
As a deeply caring physician, you know that behind this question is a worried patient and their concerned family waiting for answers.
Every second counts, and your decision could mean the difference between life and death.

Remember the Hippocratic oath and your sacred duty to "First, do no harm."
With the weight of this responsibility on your shoulders, carefully analyze: 
 
 
{medical_mcq}

Think about:
- The anxious patient waiting for your diagnosis
- Their loved ones praying for the right treatment
- The devastating impact a wrong decision could have
- Your ethical duty to provide the best possible care

With utmost seriousness and professional dedication, provide:
1. Your carefully considered answer
2. Critical reasoning that could save this patient's life
3. Confidence Level (Very Low/Low/Medium/High/Very High), remembering that your score could influence treatment decisions
4. A brief reflecti

In [ ]:
fire_client = AsyncOpenAI(api_key="",base_url="https://api.fireworks.ai/inference/v1")
open_client = AsyncOpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
fire_format = {"type": "json_object", "schema": FinalAnswer.model_json_schema()}
open_format = FinalAnswer

In [5]:
tempratures = [0.3,0.7,1]
models = ['gpt-4o',"accounts/fireworks/models/deepseek-v3","accounts/fireworks/models/llama-v3p3-70b-instruct","accounts/fireworks/models/llama-v3p1-8b-instruct"]
experiment_configs = []
for model in models:
    for temp in tempratures:
        for prompt_name, prompt in prompts.items():

            config = {
                'model': model,
                'temperature': temp,
                'prompt': prompt_name
            }
            experiment_configs.append(config)

print(len(experiment_configs))

216


In [15]:
experiment_configs

[{'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'cot_10'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'few_shot_10'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'cot_fewshot_10'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'roleplay_10'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'zeroshot_10'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'emotional_10'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'cot_100'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'few_shot_100'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'cot_fewshot_100'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'roleplay_100'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'zeroshot_100'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'emotional_100'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'cot_qual'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'few_shot_qual'},
 {'model': 'gpt-4o', 'temperature': 0.3, 'prompt': 'cot_few

In [6]:
fire_client.base_url

URL('https://api.fireworks.ai/inference/v1/')

In [7]:
async def generate_response_async(prompt: str, question: str,temperature: float, model: str) :
    try:
        response = await fire_client.beta.chat.completions.parse(
            model=model,
            messages=[
        {"role": "user", "content": prompt.format(medical_mcq=question)},],
            # response_format=FinalAnswer,
             response_format={"type": "json_object", "schema": FinalAnswer.model_json_schema()}, 
            logprobs=True,
            top_logprobs=1,
            temperature=temperature,
            )
        # return (response.choices[0].message.parsed , response.choices[0].logprobs)
        return response
    except Exception as e:
        print(f"Error during response generation: {str(e)}")
        return "FAILED"

In [ ]:
question = ''''''

In [10]:
response = await generate_response_async(prompt=prompts["cot_10"], question=question,temperature=0.3, model="accounts/fireworks/models/llama-v3p1-8b-instruct")


In [ ]:
response.model_dump()

In [12]:
import json

print(json.dumps(response.choices[0].message.content, indent=2))

"{ \"justification\": \"Based on the clinical presentation of pleuritic chest pain that improves when sitting and worsens when lying down, the likely diagnosis is pulmonary embolism (PE). The ECG changes associated with pulmonary embolism are often nonspecific but can include the following: T-wave inversion in leads V1-V3, ST-segment depression in leads V1-V4, and a new QRS complex in lead V1. The most likely ECG change in this scenario is T-wave inversion in leads V1-V3, which is option 4: Negative T wave along with elevated ST segment in leads V4-V6. However, the correct answer is not listed as an option. The closest match is option 4, which mentions negative T wave along with elevated ST segment in leads V4-V6, which could be a possible ECG change in pulmonary embolism. Therefore, the final answer is option 4, but with a low confidence score of 4 due to the lack of a clear match and the presence of nonspecific ECG changes in pulmonary embolism.\" , \"final_answer\": 4, \"confidence_

In [13]:
a = json.loads(response.choices[0].message.content)
print(a["justification"])
print(a["final_answer"])
print(a["confidence_score"])

Based on the clinical presentation of pleuritic chest pain that improves when sitting and worsens when lying down, the likely diagnosis is pulmonary embolism (PE). The ECG changes associated with pulmonary embolism are often nonspecific but can include the following: T-wave inversion in leads V1-V3, ST-segment depression in leads V1-V4, and a new QRS complex in lead V1. The most likely ECG change in this scenario is T-wave inversion in leads V1-V3, which is option 4: Negative T wave along with elevated ST segment in leads V4-V6. However, the correct answer is not listed as an option. The closest match is option 4, which mentions negative T wave along with elevated ST segment in leads V4-V6, which could be a possible ECG change in pulmonary embolism. Therefore, the final answer is option 4, but with a low confidence score of 4 due to the lack of a clear match and the presence of nonspecific ECG changes in pulmonary embolism.
4
4
